[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Asterios7/dlib-wrapper/blob/main/notebooks/examples-colab.ipynb)


# Examples for dlib-wrapper package

## Notebook prerequisites

In [ ]:
# Install the dlib-wrapper package from github
!pip install git+https://github.com/Asterios7/dlib-wrapper.git -q

In [ ]:
# Imports
import os
import requests
import zipfile
from dlib_wrapper import  dlibFaceProcessor
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def euclidean_distance(vec1: np.ndarray, vec2: np.ndarray) -> float:
    return np.linalg.norm(vec1 - vec2)

In [ ]:
# Fetch sample images
data_path = "./data"
data_url = "https://github.com/Asterios7/dlib-wrapper/raw/main/data/data.zip"

if os.path.isdir(data_path):
    print(f"{data_path} directory exists.")
else:
    print(f"{data_path} directory does not exist, creating...")
    os.makedirs(data_path, exist_ok=True)

    with open('data.zip', 'wb') as f:
        print("Downloading sample image data...")
        request = requests.get(data_url)
        f.write(request.content)

    with zipfile.ZipFile("data.zip", "r") as zip_ref:
        print("Unzipping data...")
        zip_ref.extractall("data")
        os.remove("data.zip")

In [ ]:
# Open and plot sample image
img = Image.open("./data/pulp-fiction.jpg")
img = np.array(img)
plt.imshow(img)
plt.title("Vincent and Jules casually conducting business")
plt.axis('off');

## Using dlib-wrapper

In [ ]:
# Create a `dlibFaceProcessor` instance
face_processor = dlibFaceProcessor()

### Image to face embeddings (option 1)

`image` &rarr; `embeddings`

Using the `dlibFaceProcessor.detect_and_encode_faces`

In [ ]:
# Create face embeddings from image
embeddings = face_processor.detect_and_encode_faces(img)
print(f"Number of faces detected: {len(embeddings)}")
print(f"Number of embeddings per face: {len(embeddings[0])}")

### Image to face embeddings (option 2)

`image` &rarr; `boxes` &rarr; `landmarks` &rarr; `aligned_faces` &rarr; `embeddings`

Step by step implementation for generating face embeddings. Use this if besides creating the embeddings you also want to:
- utilize face landmarks or
- extract the faces (aligned)

In [ ]:
# Detect faces, get face boxes
boxes = face_processor.detect_faces(img)
# Extract face landmarks
shapes = face_processor.get_shapes(img, boxes)
# Align faces
aligned_faces = face_processor.align_faces(img, shapes)
# Extract face embeddings
embeddings = face_processor.encode_faces(aligned_faces)

In [ ]:
# Plot face landmarks
fig, ax = plt.subplots(figsize=(8,4))
ax.imshow(img)
for landmarks in shapes:
    # Plot landmarks on the face
    for n in range(0, 5):  # 5 face landmarks
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        plt.scatter(x, y, color='green', s=5)
ax.axis("off");

In [ ]:
# Plot faces after detection and alignment
fig, axes = plt.subplots(1, len(aligned_faces), figsize=(8, 4))
for i, face in enumerate(aligned_faces):
    axes[i].imshow(face)
    axes[i].axis('off')

In [ ]:
# Check embeddings
print(f"Number of faces detected: {len(boxes)}")
print(f"Number of embeddings per face: {len(embeddings[0])}")

## Face recognition

When using a distance threshold of 0.6, the dlib model obtains an accuracy > 99% on the standard LFW face recognition benchmark.

More at: http://dlib.net/dnn_face_recognition_ex.cpp.html

In [ ]:
threshold = 0.6
# Load images
img1 = Image.open("./data/pulp-fiction.jpg")
img2 = Image.open("./data/sam-j.jpg")

# Convert image to np.ndarray
img1, img2 =  np.array(img1), np.array(img2)

# Plot images
fig, axes = plt.subplots(1, len([img1, img2]), figsize=(10, 3))
for i, image in enumerate([img1, img2]):
    axes[i].imshow(image)
    axes[i].set_title(f"img{i+1}")
    axes[i].axis('off')

In [ ]:
# Instantiate dlibFaceProcessor (repeating this line for completeness)
face_processor = dlibFaceProcessor()

In [ ]:
# Detect faces, get face boxes
boxes1 = face_processor.detect_faces(img1)
boxes2 = face_processor.detect_faces(img2)
# Extract face landmarks
shapes1 = face_processor.get_shapes(img1, boxes1)
shapes2 = face_processor.get_shapes(img2, boxes2)
# Align faces
aligned_faces1 = face_processor.align_faces(img1, shapes1)
aligned_faces2 = face_processor.align_faces(img2, shapes2)
# Extract face embeddings
embeddings1 = face_processor.encode_faces(aligned_faces1)
embeddings2 = face_processor.encode_faces(aligned_faces2)

print(f"\nNumber of faces detected on img1: {len(embeddings1)}")
print(f"Number of faces detected on img2: {len(embeddings2)}")

In [ ]:
# Compare the two faces of img1 to the face of the img2

for i in range(len(embeddings1)):

    distance = euclidean_distance(embeddings1[i], embeddings2[0])

    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes[0].imshow(aligned_faces1[i])
    axes[0].axis('off')
    axes[1].imshow(aligned_faces2[0])
    axes[1].axis('off')
    if distance > threshold:
        fig.suptitle(f'Distance: {distance:.3f} => No Match');
    else:
        fig.suptitle(f'Distance: {distance:.3f} => Match');